## Лабораторна робота №8
__Варіант 1__

Завдання до лабораторної роботи:
1. Створити кілька своїх прикладів у форматі json за тематикою варіанту (англійською або українською мовою) для розпізнавання нового типу сутностей (обрати самостійно). Створити програму, що додає ці приклади до існуючої моделі spaCy, навчає модель. Продемонструвати роботу.
2. Застосувати компонент TextCategorizer для визначення намірів. Дані для навчання за тематикою варіанту обрати самостійно або скористатись вказаним файлом (utterance містить висловлювання, intent - намір). Дані файли містять приклади діалогів користувачів з системою-помічником за певною тематикою, наприклад, замовлення квитків і т.д. Навчити компонент та продемонструвати роботу.

Варіант 1.
Тематика: музика. Файл music.json.

1. Створити кілька своїх прикладів у форматі json за тематикою варіанту (англійською або українською мовою) для розпізнавання нового типу сутностей (обрати самостійно). Створити програму, що додає ці приклади до існуючої моделі spaCy, навчає модель. Продемонструвати роботу.

[
    {
        "text": "Rammstein is a German Neue Deutsche Härte band formed in Berlin in 1994.",
        "entities": [
            {"start": 0, "end": 9, "label": "MUSIC_ARTIST"}
        ]
    },
    {
        "text": "Rammstein's album Mutter includes some of their most famous songs.",
        "entities": [
            {"start": 0, "end": 18, "label": "MUSIC_ALBUM"},
            {"start": 0, "end": 9, "label": "MUSIC_ARTIST"}
        ]
    },
    {
        "text": "Other bands in the Neue Deutsche Härte genre include Oomph! and Eisbrecher.",
        "entities": [
            {"start": 54, "end": 60, "label": "MUSIC_ARTIST"},
            {"start": 65, "end": 75, "label": "MUSIC_ARTIST"}
        ]
    }
]


In [ ]:
import spacy
import random
from spacy.training.example import Example

In [ ]:
nlp = spacy.load("en_core_web_md")

trainset = [
    ("Rammstein is a German Neue Deutsche Härte band formed in Berlin in 1994.", {"entities": [(0, 9, "MUSIC_ARTIST")]}),
    ("Rammstein's album Mutter includes some of their most famous songs.", {"entities": [(0, 9, "MUSIC_ARTIST"), (18, 24, "MUSIC_ALBUM")]}),
    ("Other bands in the Neue Deutsche Härte genre include Oomph! and Eisbrecher.", {"entities": [(54, 60, "MUSIC_ARTIST"), (65, 75, "MUSIC_ARTIST")]}),
]

epochs = 20

other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):
    optimizer = nlp.create_optimizer()
    for i in range(epochs):
        random.shuffle(trainset)
        for text, annotations in trainset:
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, annotations)
            nlp.update([example], sgd=optimizer)

ner = nlp.get_pipe("ner")
ner.to_disk("new_ner")

In [ ]:
doc= nlp("Rammstein's album Mutter was released in 2001.")
print([(ent.text, ent.label_) for ent in doc.ents])

[('Rammstein', 'MUSIC_ARTIST'), ('Mutter', 'MUSIC_ALBUM')]


2. Застосувати компонент TextCategorizer для визначення намірів. Дані для навчання за тематикою варіанту обрати самостійно або скористатись вказаним файлом (utterance містить висловлювання, intent - намір). Дані файли містять приклади діалогів користувачів з системою-помічником за певною тематикою, наприклад, замовлення квитків і т.д. Навчити компонент та продемонструвати роботу.

In [ ]:
import json
from typing import List, Tuple

def load_data(file_path: str) -> List[Tuple[str, dict]]:
    with open(file_path, 'r') as f:
        data = json.load(f)
    training_data = []
    for dialogue in data:
        for turn in dialogue['turns']:
            if turn['speaker'] == 'USER':
                for frame in turn['frames']:
                    if frame['state']['active_intent']:
                        training_data.append(
                            (turn['utterance'], {'cats': {frame['state']['active_intent']: 1.0}})
                        )
    return training_data

file_path = 'music.json'
TRAIN_DATA = load_data(file_path)

In [ ]:
import spacy
from spacy.util import minibatch, compounding
from spacy.training import Example

In [ ]:
# без заданих міток/ without pre-defined labels

# Load a spacy model.
nlp = spacy.blank('en')

# Add TextCategorizer to the pipeline
textcat = nlp.add_pipe("textcat")

# Add labels to the TextCategorizer
for _, annotations in TRAIN_DATA:
    for label in annotations['cats'].keys():
        textcat.add_label(label)

# Train the TextCategorizer
optimizer = nlp.initialize()
batch_sizes = compounding(4.0, 32.0, 1.001)
for i in range(10):  # Number of training iterations
    loss = {}
    for text, annotations in TRAIN_DATA:
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)
        nlp.update([example], sgd=optimizer, losses=loss)
    print(f"Loss: {loss}")  # Print the training loss

Loss: {'textcat': 50.9850091059602}
Loss: {'textcat': 32.58437335263054}
Loss: {'textcat': 28.826258899176327}
Loss: {'textcat': 24.761861798954165}
Loss: {'textcat': 22.341238014918765}
Loss: {'textcat': 20.631518462545355}
Loss: {'textcat': 20.029475309292806}
Loss: {'textcat': 18.37929235322587}
Loss: {'textcat': 16.811501895722923}
Loss: {'textcat': 18.931701475994082}


In [ ]:
test_text = "I want to listen to some songs."
doc = nlp(test_text)
print(doc.cats)  # Outputs the probability score for each category

{'LookupMusic': 0.705403745174408, 'PlayMedia': 4.3366049794713035e-05, 'NONE': 0.000829760916531086, 'LookupSong': 0.28418806195259094, 'PlaySong': 0.00953500997275114}
